In [1]:
import mediapipe as mp
import cv2
import time
import numpy as np

In [2]:
mpDraw = mp.solutions.drawing_utils
mpPoses = mp.solutions.pose

mpDraw_point = mpDraw.DrawingSpec((255, 255, 255), 3 , 3)
mpDraw_line = mpDraw.DrawingSpec((0, 255, 255), 2, 2)

poses = mpPoses.Pose(min_detection_confidence=0.5, # 最小偵測置信度
                    min_tracking_confidence=0.5)  # 最小追蹤置信度

## 身體骨架點

![](https://google.github.io/mediapipe/images/mobile/pose_tracking_full_body_landmarks.png)

In [3]:
cap = cv2.VideoCapture(0)

while cap.isOpened():
    hasFrame, frame = cap.read()                         # 讀取攝影機的有無訊息與當前畫面
    if not hasFrame: break                               # 攝影機無畫面時關閉
    
    (imgHeight, imgWidth, _) = frame.shape               # 影像形狀的寬高(於後骨架需要正規化回來計算所需)
    imgRGB = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)      # 彩色影像GBR通道轉換為RGB通道

    result = poses.process(imgRGB)                        # Mediapipe身體骨架預測結果
    if result.pose_landmarks:
        for i, lm in enumerate(result.pose_landmarks.landmark):
            mpDraw.draw_landmarks(frame, result.pose_landmarks, mpPoses.POSE_CONNECTIONS, mpDraw_point, mpDraw_line)
        
            # 身體骨架點正規化還原
            xPos = int(lm.x * imgWidth)
            yPos = int(lm.y * imgHeight)
            
            # 影像上放置文字
            cv2.putText(frame, str(i), (xPos-25, yPos+5), cv2.FONT_HERSHEY_SIMPLEX, 0.4, (0,0,255), 2)
                
            #if i==8:
            #    cv2.circle(frame, (xPos, yPos), 8, (255, 0, 0), cv2.FILLED) 

    cv2.imshow("Hand Tracking", frame)

    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()